In [1]:
## 1. الإعدادات، استيراد المكتبات، وتجهيز البيانات

import os
import zipfile
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, classification_report
from google.colab import drive

# 1. ربط Google Drive
print("جاري ربط Google Drive...")
drive.mount('/content/drive')

# 2. تحديد المسارات الثابتة
DRIVE_ROOT = '/content/drive/MyDrive'
PROJECT_FOLDER = 'SmartHospital_SkinClinic'
ZIP_FILE_NAME = 'skin_cancer_dataset.zip'
EXTRACT_FOLDER = 'skin_data_clinic_prep'
zip_path = os.path.join(DRIVE_ROOT, PROJECT_FOLDER, ZIP_FILE_NAME)
extract_path = f'/content/{EXTRACT_FOLDER}'

# 3. فك ضغط البيانات (لضمان وجود البيانات)
print("\nجاري فك ضغط البيانات...")
if not os.path.exists(extract_path):
    os.makedirs(extract_path)
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("تم فك ضغط البيانات بنجاح.")
except FileNotFoundError:
    print(f"خطأ: لم يتم العثور على الملف المضغوط في المسار: {zip_path}")

# 4. **ضبط المسار النهائي (DATA_DIR) بناءً على الهيكلية الفعلية**
INNER_FOLDER_NAME = 'skin-cancer-malignant-vs-benign' # اسم المجلد الداخلي
DATA_DIR = os.path.join(extract_path, INNER_FOLDER_NAME)

# التحقق من المسار وتعديله إن لزم (لتفادي أي خطأ)
if not os.path.exists(os.path.join(DATA_DIR, 'train')):
    # إذا لم يكن المجلد الداخلي موجودًا (أي أن الهيكلية كانت مباشرة)، نعود للمسار الأساسي
    DATA_DIR = extract_path
    print("تنبيه: تم استخدام المسار الأساسي (بدون مجلد داخلي).")

train_path = os.path.join(DATA_DIR, 'train')
test_path = os.path.join(DATA_DIR, 'test')

# التحقق النهائي من وجود مجلد train قبل إنشاء الـ Generator
if not os.path.exists(train_path):
    raise FileNotFoundError(f"خطأ قاتل: مجلد التدريب غير موجود في: {train_path}. يرجى التحقق من هيكل ملف ZIP.")



IMAGE_SIZE = (150, 150)
BATCH_SIZE = 32
MODEL_SAVE_DIR = os.path.join(DRIVE_ROOT, PROJECT_FOLDER)


train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip=True, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='binary', shuffle=True
)
test_generator = test_datagen.flow_from_directory(
    test_path, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='binary', shuffle=False
)

counter = train_generator.classes
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(counter),
    y=counter
)
class_weights_dict = dict(enumerate(class_weights))
print("\nتم حساب أوزان التصنيفات (Class Weights) بنجاح وجاهزية الـ Generators.")

جاري ربط Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

جاري فك ضغط البيانات...
تم فك ضغط البيانات بنجاح.
تنبيه: تم استخدام المسار الأساسي (بدون مجلد داخلي).
Found 2637 images belonging to 2 classes.
Found 660 images belonging to 2 classes.

تم حساب أوزان التصنيفات (Class Weights) بنجاح وجاهزية الـ Generators.


In [7]:

from tensorflow.keras.applications import MobileNetV2

base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
)

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')]
)

checkpoint_filepath = os.path.join(MODEL_SAVE_DIR, 'best_model_weights.weights.h5')
if not os.path.exists(MODEL_SAVE_DIR):
    os.makedirs(MODEL_SAVE_DIR)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

print("\n--- هيكلة MobileNetV2 جاهزة للتدريب السريع ---")

/tmp/ipython-input-608005228.py:7: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step

--- هيكلة MobileNetV2 جاهزة للتدريب السريع ---


In [6]:
# ## 3. تدريب النموذج وتطبيق التقنيات المتقدمة

# EPOCHS = 15 # عدد Epochs كبير والاعتماد على EarlyStopping للإيقاف

# print("\nبدء تدريب النموذج بتطبيق Class Weights و Callbacks...")
# history = model.fit(
#     train_generator,
#     steps_per_epoch=train_generator.samples // BATCH_SIZE,
#     epochs=EPOCHS,
#     validation_data=test_generator,
#     validation_steps=test_generator.samples // BATCH_SIZE,
#     callbacks=[early_stopping, model_checkpoint_callback],
#     class_weight=class_weights_dict # تطبيق أوزان التصنيفات لضبط الـ Imbalance
# )

In [9]:
## 3. التدريب الأولي والتعلم المُنقّح (Fine-Tuning)
EPOCHS_INITIAL = 10

print("\nبدء المرحلة 1: التدريب الأولي السريع...")
history_initial = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE // 2,
    epochs=EPOCHS_INITIAL,
    validation_data=test_generator,
    validation_steps=test_generator.samples // BATCH_SIZE,
    callbacks=[early_stopping],
    class_weight=class_weights_dict
)

print("\n✅ انتهاء التدريب الأولي. بدء المرحلة 2: التعلم المُنقّح لزيادة الدقة والـ Recall...")

base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=0.00001),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')]
)

EPOCHS_FINE = 15
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS_FINE,
    validation_data=test_generator,
    validation_steps=test_generator.samples // BATCH_SIZE,
    callbacks=[early_stopping, model_checkpoint_callback],
    class_weight=class_weights_dict
)


بدء المرحلة 1: التدريب الأولي السريع...
Epoch 1/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 61s 1s/step - accuracy: 0.8161 - loss: 0.3974 - precision: 0.7850 - recall: 0.8208 - val_accuracy: 0.8375 - val_loss: 0.3832 - val_precision: 0.7953 - val_recall: 0.8464
Epoch 2/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.8098 - loss: 0.4030 - precision: 0.7860 - recall: 0.8335 - val_accuracy: 0.8391 - val_loss: 0.3934 - val_precision: 0.8041 - val_recall: 0.8357
Epoch 3/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 21s 505ms/step - accuracy: 0.8438 - loss: 0.4393 - precision: 0.8421 - recall: 0.8889 - val_accuracy: 0.8156 - val_loss: 0.4015 - val_precision: 0.8092 - val_recall: 0.7571
Epoch 4/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 61s 2s/step - accuracy: 0.7973 - loss: 0.4261 - precision: 0.7825 - recall: 0.7631 - val_accuracy: 0.8125 - val_loss: 0.4139 - val_precision: 0.7312 - val_recall: 0.9036
Epoch 5/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.8072 - loss: 0.3845 - precision: 0.7620 - recall: 0.8745 - v

In [10]:
## 4. التقييم الشاملة الـ Recall، وحفظ النموذج النهائي

test_generator.reset()
Y_pred = model.predict(test_generator, steps=test_generator.samples // BATCH_SIZE + 1)


THRESHOLD = 0.42

y_pred = np.where(Y_pred > THRESHOLD, 1, 0)
y_true = test_generator.classes[:len(y_pred)]


21/21 ━━━━━━━━━━━━━━━━━━━━ 75s 4s/step


In [11]:


from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

model.load_weights(checkpoint_filepath)
print("\nتم تحميل أفضل الأوزان التي حققت أعلى دقة تحقق بعد مرحلة Fine-Tuning.")

test_generator.reset()
Y_pred = model.predict(test_generator, steps=test_generator.samples // BATCH_SIZE + 1)

THRESHOLD = 0.42

y_pred = np.where(Y_pred > THRESHOLD, 1, 0)
y_true = test_generator.classes[:len(y_pred)]

print("\n--- تقرير التصنيف الشامل (Classification Report) ---")
# يوضح الـ Accuracy والـ Precision والـ Recall لكل تصنيف (Benign/Malignant)
report = classification_report(y_true, y_pred, target_names=['Benign (0)', 'Malignant (1)'], output_dict=True)
print(classification_report(y_true, y_pred, target_names=['Benign (0)', 'Malignant (1)']))

cm = confusion_matrix(y_true, y_pred)
print("\n--- مصفوفة الارتباك (Confusion Matrix) ---")
print(cm)

# 4. استخراج المقاييس النهائية للتقرير ولوحة التحكم
accuracy = report['accuracy']
f1_score = report['weighted avg']['f1-score']
precision_malignant = report['Malignant (1)']['precision']
recall_malignant = report['Malignant (1)']['recall']

print(f"\n=======================================================")
print(f"✅ Accuracy النهائي: {accuracy:.4f}")
print(f"✅ Recall (Malignant): {recall_malignant:.4f} (مقياس الكشف عن السرطان)")
print(f"=======================================================")
print(f"F1 Score النهائي (Weighted): {f1_score:.4f}")
print(f"Precision (Malignant): {precision_malignant:.4f}")


# 5. حفظ النموذج النهائي (المُخرج رقم 9)
FINAL_MODEL_SAVE_PATH = os.path.join(MODEL_SAVE_DIR, 'skin_cancer_final_model.h5')
model.save(FINAL_MODEL_SAVE_PATH)
print(f"\nتم حفظ النموذج النهائي (skin_cancer_final_model.h5) بنجاح في Drive.")


تم تحميل أفضل الأوزان التي حققت أعلى دقة تحقق بعد مرحلة Fine-Tuning.
21/21 ━━━━━━━━━━━━━━━━━━━━ 18s 786ms/step



--- تقرير التصنيف الشامل (Classification Report) ---
               precision    recall  f1-score   support

   Benign (0)       0.91      0.80      0.85       360
Malignant (1)       0.79      0.90      0.84       300

     accuracy                           0.85       660
    macro avg       0.85      0.85      0.85       660
 weighted avg       0.85      0.85      0.85       660


--- مصفوفة الارتباك (Confusion Matrix) ---
[[289  71]
 [ 30 270]]

✅ Accuracy النهائي: 0.8470
✅ Recall (Malignant): 0.9000 (مقياس الكشف عن السرطان)
F1 Score النهائي (Weighted): 0.8472
Precision (Malignant): 0.7918

تم حفظ النموذج النهائي (skin_cancer_final_model.h5) بنجاح في Drive.
